In [26]:
# IMPORTS

import numpy as np
import pandas as pd
import re
import statistics as stats

import mysql.connector
import sqlalchemy
import getpass

In [27]:
# FUNCTIONS

# Describe

def describe_col(df, col):
    print(f'{upper_spaced(col)} ({col})')
    print(f'\nGeneral stats')
    print(df[col].describe())
    print(f'\nValue distribution')
    print(df[col].value_counts())
    
def distrib_missing_values(df, cols):
    d_missing_values = {'id': [], 'count_missing_values': []}
    for i in df.index:
        d_missing_values['id'].append(i)
        d_missing_values['count_missing_values'].append(df.loc[i, cols].isna().sum())
    df_missing_values = pd.DataFrame.from_dict(d_missing_values)
    print(df_missing_values['count_missing_values'].value_counts())
    return d_missing_values
    
    
def describe_unique_values(df, col):
    print(f'{upper_spaced(col)} ({col})')
    print(f'\nUnique values')
    print(sorted(df[col].unique()))
    
def upper_spaced(my_str):
    return re.sub(r"(\w)([A-Z])", r"\1 \2", my_str).upper()
    
# Filter

def get_null_cols(df, cols):
    null_vals_by_col = df[cols].isnull().sum()
    l_null_cols = list(null_vals_by_col[null_vals_by_col > 0].index)
    return l_null_cols

def get_low_variance_cols(df, percentile = 90):
    low_variance = []
    for col in df._get_numeric_data():
        minimum = min(df[col])
        upper_bound = np.percentile(df[col], percentile)
        if upper_bound == minimum:
            low_variance.append(col)
    return low_variance

def get_numeric_columns(df):
    return list(df.select_dtypes('number'))

def get_nonnumeric_columns(df):
    return list(df.select_dtypes('object'))

# Homogeneise

def homogeneise_column_names(df, replace_dict):
    for old, new in replace_dict.items():
        df.columns = df.columns.str.replace(old, new)
        
def homogeneise_values(df, replace_dict):
    for old, new in replace_dict.items():
        df.replace(old, new, inplace = True)

# Fill

def fill_missing_vals(df, d_filling_meths):
    for meth, l_cols in d_filling_meths.items():
        for col in l_cols:
            print(f'Filling: {col}...')
            df[col].fillna(meth(df[col]), inplace = True)
            
# Drop

def drop_too_many_missing_values(df, d_missing_values, max_missing_values):
    rows_to_drop = [d_missing_values['id'][i] for i in d_missing_values['id'] if d_missing_values['count_missing_values'][i] > max_missing_values]
    print(f'Dropping {len(rows_to_drop)} rows ({len(rows_to_drop) / len(df.index):.1%} of total): {rows_to_drop}.')
    df.drop(df.index[rows_to_drop], inplace = True)
    
def drop_duplicates_from_subset(df, cols_subset):
    before = len(df)
    df[cols_subset].drop_duplicates(inplace = True)
    after = len(df)
    nb_drop = before - after
    print(f'Number of duplicate records dropped: {nb_drop} ({nb_drop / len(df): .1%} of total)')
    
# Other

def zerofy(value):
    return 0

def emptyfy(string):
    return ""

def unknownfy(string):
    return "unknown"

In [28]:
# DATA IMPORT
df = pd.read_excel('../data/2 -Entrepreneurial competency in university students.xlsx')
df1 = df.copy()

In [29]:
# DESCRIPTION OF ORIGINAL DATASET
df1.head(5)

,EducationSector,Target IndividualProject,Age,Gender,City,Influenced,Perseverance,DesireToTakeInitiative,Competitiveness,SelfReliance,StrongNeedToAchieve,SelfConfidence,GoodPhysicalHealth,MentalDisorder,KeyTraits,ReasonsForLack,Target-ent_competency
0,Engineering Sciences,No,19.0,Male,Yes,No,2.0,2.0,3.0,3.0,2.0,2.0,3.0,Yes,Passion,NaN,1
1,Engineering Sciences,Yes,22.0,Male,No,Yes,3.0,3.0,3.0,4.0,4.0,3.0,4.0,Yes,Vision,Just not interested! (Want to work in the corp...,0
2,Engineering Sciences,No,18.0,Male,Yes,No,3.0,4.0,3.0,3.0,3.0,4.0,4.0,No,Passion,Not willing to start a venture in India and wa...,0
3,Engineering Sciences,Yes,20.0,Male,Yes,Yes,3.0,3.0,3.0,3.0,4.0,3.0,3.0,No,Rrresilience,Not able to take a Financial Risk,0
4,Engineering Sciences,Yes,19.0,Male,Yes,Yes,2.0,3.0,3.0,3.0,4.0,3.0,2.0,Yes,Vision,NaN,1


In [30]:
df1.describe()

,Age,Perseverance,DesireToTakeInitiative,Competitiveness,SelfReliance,StrongNeedToAchieve,SelfConfidence,GoodPhysicalHealth,Target-ent_competency
count,205.000000,202.000000,208.000000,210.000000,197.000000,211.000000,212.000000,212.000000,219.000000
mean,19.756098,3.346535,3.591346,3.571429,3.705584,3.914692,3.566038,3.566038,0.415525
std,1.324366,1.001845,1.163587,1.118416,1.066474,1.033866,1.122952,1.105941,0.493941
min,17.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,19.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,0.000000
50%,20.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,0.000000
75%,20.000000,4.000000,5.000000,4.000000,5.000000,5.000000,4.000000,4.000000,1.000000
max,26.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1.000000


In [31]:
# HOMOGENEISE COLUMN NAMES
replace_dict_colnames = {
                    ' ': '',
                    '-': '_'
                    }

homogeneise_column_names(df1, replace_dict_colnames)

In [32]:
# DROP DUPLICATES
l_null_numeric_cols = get_null_cols(df1, get_numeric_columns(df1))
cols_subset = l_null_numeric_cols + ['EducationSector', 'Gender']
drop_duplicates_from_subset(df1, cols_subset)

Number of duplicate records dropped: 0 ( 0.0% of total)


/Users/ericmartinet/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [33]:
# CHECK LOW VARIANCE COLUMNS
percentile_filter = 90
l_low_variance_cols = get_low_variance_cols(df1, percentile_filter)
print(l_low_variance_cols)

[]


In [34]:
# MISSING NUMERIC VALUES
# Check rows with missing numeric values
d_missing_numeric_values = distrib_missing_values(df1, l_null_numeric_cols)

0    181
1     18
2     11
7      7
3      2
Name: count_missing_values, dtype: int64


In [35]:
# Drop rows with too many missing numeric values
max_missing_values = 2
drop_too_many_missing_values(df1, d_missing_numeric_values, max_missing_values)

Dropping 9 rows (4.1% of total): [76, 82, 198, 199, 200, 201, 202, 203, 204].


In [36]:
# Describe each numeric column with missing values
for col in l_null_numeric_cols:
    describe_col(df1, col)
    print('\n--------------------')

AGE (Age)

General stats
count    196.000000
mean      19.780612
std        1.335062
min       17.000000
25%       19.000000
50%       20.000000
75%       20.000000
max       26.000000
Name: Age, dtype: float64

Value distribution
20.0    67
19.0    61
21.0    27
18.0    18
22.0    12
17.0     6
23.0     2
26.0     1
24.0     1
25.0     1
Name: Age, dtype: int64

--------------------
PERSEVERANCE (Perseverance)

General stats
count    201.000000
mean       3.348259
std        1.004047
min        1.000000
25%        3.000000
50%        3.000000
75%        4.000000
max        5.000000
Name: Perseverance, dtype: float64

Value distribution
4.0    70
3.0    65
2.0    36
5.0    24
1.0     6
Name: Perseverance, dtype: int64

--------------------
DESIRE TO TAKE INITIATIVE (DesireToTakeInitiative)

General stats
count    207.000000
mean       3.584541
std        1.162251
min        1.000000
25%        3.000000
50%        4.000000
75%        5.000000
max        5.000000
Name: DesireToTakeInitia

In [37]:
# Fill missing values of numeric columns with a specific method
d_filling_numeric_meths = {
    zerofy: [],
    np.mean: [],
    np.nanmedian: ['Age', 'Perseverance', 'DesireToTakeInitiative', 'Competitiveness', 'SelfReliance', 'StrongNeedToAchieve', 'SelfConfidence', 'GoodPhysicalHealth'],
    }

fill_missing_vals(df1, d_filling_numeric_meths)

Filling: Age...
Filling: Perseverance...
Filling: DesireToTakeInitiative...
Filling: Competitiveness...
Filling: SelfReliance...
Filling: StrongNeedToAchieve...
Filling: SelfConfidence...
Filling: GoodPhysicalHealth...


In [38]:
# MISSING NON NUMERIC VALUES
# TBC
l_null_nonnumeric_cols = get_null_cols(df1, get_nonnumeric_columns(df1))

# ReasonsForLack can be empty
try:
    l_null_nonnumeric_cols.remove('ReasonsForLack')
except:
    pass

d_missing_nonnumeric_values = distrib_missing_values(df1, l_null_nonnumeric_cols)

0    199
1     11
Name: count_missing_values, dtype: int64


In [39]:
# Describe each non numeric column with missing values
for col in l_null_nonnumeric_cols:
    describe_col(df1, col)
    print('\n--------------------')

MENTAL DISORDER (MentalDisorder)

General stats
count     199
unique      2
top        No
freq      142
Name: MentalDisorder, dtype: object

Value distribution
No     142
Yes     57
Name: MentalDisorder, dtype: int64

--------------------


In [40]:
d_filling_nonnumeric_meths = {
    emptyfy: [],
    unknownfy: ['MentalDisorder'],
    stats.mode: []
    }

fill_missing_vals(df1, d_filling_nonnumeric_meths)

Filling: MentalDisorder...


In [41]:
# Now all values are filled and we have dropped some rows
# So we can reset the index before correcting and homogeneising the values
df1.reset_index(inplace = True)

In [42]:
# INCORRECT VALUES
# ReasonsForLack must be NaN if Target-ent_competency = 1

df1.loc[(df1['Target_ent_competency'] == 1) & (df1['ReasonsForLack'] == 1), 'index'].count()

0

In [43]:
# No incorrect values

In [44]:
# HOMOGENEISE VALUES

# Analyse all columns but ReasonsForLack and Index

cols_homogeneity_exclude = ['level_0', 'index', 'ReasonsForLack']
cols_homogeneity = [col for col in list(df1.columns) if col not in cols_homogeneity_exclude]

for col in cols_homogeneity:
    describe_unique_values(df1, col)
    print('\n--------------------')

EDUCATION SECTOR (EducationSector)

Unique values
['Art, Music or Design', 'Economic Sciences, Business Studies, Commerce and Law', 'Engineering Sciences', 'Humanities and Social Sciences', 'Language and Cultural Studies', 'Mathematics or Natural Sciences', 'Medicine, Health Sciences', 'Others', 'Teaching Degree (e.g., B.Ed)']

--------------------
TARGET INDIVIDUAL PROJECT (TargetIndividualProject)

Unique values
['No', 'Yes']

--------------------
AGE (Age)

Unique values
[17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0]

--------------------
GENDER (Gender)

Unique values
['Female', 'Male', 'male']

--------------------
CITY (City)

Unique values
['No', 'Yes', 'unknown']

--------------------
INFLUENCED (Influenced)

Unique values
['No', 'Yes', 'yes']

--------------------
PERSEVERANCE (Perseverance)

Unique values
[1.0, 2.0, 3.0, 4.0, 5.0]

--------------------
DESIRE TO TAKE INITIATIVE (DesireToTakeInitiative)

Unique values
[1.0, 2.0, 3.0, 4.0, 5.0]

------------------

In [45]:
# Homogeneise all values

replace_dict_values = {
                        'male': 'Male',
                        'yes': 'Yes',
                        'no': 'No',
                        'passion': 'Passion',
                        'Rrresilience': 'Resilience'
                        }

homogeneise_values(df1, replace_dict_values)

In [46]:
# Specific treatment: ReasonsForLack
# We will break down the reasons and encode them as 0 / 1

# Break down ReasonsForLack into list
lst_lst_reasons = []
for reason in df1.loc[:,'ReasonsForLack']:
    lst_lst_reasons.append(re.split(', (?=[A-Z])', str(reason)))

# Get unique reasons
unique_possible_reasons = set([reason for lst_reasons in lst_lst_reasons for reason in lst_reasons])
unique_possible_reasons.remove('nan')
print(unique_possible_reasons)

{'Just not interested! (Want to work in the corporate sector, or for the government or pursue research or something else)', 'Parental Pressure', 'Lack of Knowledge', 'Not able to take a Financial Risk', 'Mental Block', 'Unwillingness to take risk', 'Academic Pressure', 'Not willing to start a venture in India and waiting for future relocation'}


In [47]:
# Create new columns for encoding, and put all values to 0
new_cols = unique_possible_reasons
df1 = df1.assign(**dict.fromkeys(new_cols, 0))

In [48]:
# Since the reset the index previously
# df1.index is from 0 to 209
# aligned with the length of lst_lst_reasons
print(df1.index)
print(len(lst_lst_reasons))

RangeIndex(start=0, stop=210, step=1)
210


In [49]:
# Encode to 1 if student has reason
for index in df1.index:
    for reason in lst_lst_reasons[index]:
        if reason in unique_possible_reasons:
            df1.loc[index, reason] = 1

In [50]:
# Rename columns
rename_column_dict = {
        'Academic Pressure':'AcademicPressure',
        'Just not interested! (Want to work in the corporate sector, or for the government or pursue research or something else)': 'NotInterested',
        'Lack of Knowledge': 'LackKnowledge', 
        'Mental Block':'MentalBlock',
        'Not able to take a Financial Risk': 'NotAbleFinancialRisk',
        'Not willing to start a venture in India and waiting for future relocation': "FutureRelocation", 
        'Parental Pressure': 'ParentalPressure',
        'Unwillingness to take risk': 'RiskAdverse'
        }

df1.rename(columns = rename_column_dict, inplace = True)

In [51]:
# DATATYPES
# Describe
df1.dtypes

index                        int64
EducationSector             object
TargetIndividualProject     object
Age                        float64
Gender                      object
City                        object
Influenced                  object
Perseverance               float64
DesireToTakeInitiative     float64
Competitiveness            float64
SelfReliance               float64
StrongNeedToAchieve        float64
SelfConfidence             float64
GoodPhysicalHealth         float64
MentalDisorder              object
KeyTraits                   object
ReasonsForLack              object
Target_ent_competency        int64
NotInterested                int64
ParentalPressure             int64
LackKnowledge                int64
NotAbleFinancialRisk         int64
MentalBlock                  int64
RiskAdverse                  int64
AcademicPressure             int64
FutureRelocation             int64
dtype: object

In [52]:
# Transform all numeric columns into integers (as they are discreet natural values)
for col in get_numeric_columns(df1):
    df1[col] = df1[col].astype('int64')

In [53]:
# Check results
df1.dtypes

index                       int64
EducationSector            object
TargetIndividualProject    object
Age                         int64
Gender                     object
City                       object
Influenced                 object
Perseverance                int64
DesireToTakeInitiative      int64
Competitiveness             int64
SelfReliance                int64
StrongNeedToAchieve         int64
SelfConfidence              int64
GoodPhysicalHealth          int64
MentalDisorder             object
KeyTraits                  object
ReasonsForLack             object
Target_ent_competency       int64
NotInterested               int64
ParentalPressure            int64
LackKnowledge               int64
NotAbleFinancialRisk        int64
MentalBlock                 int64
RiskAdverse                 int64
AcademicPressure            int64
FutureRelocation            int64
dtype: object

In [54]:
# Check final dataset
df1.head(5)

,index,EducationSector,TargetIndividualProject,Age,Gender,City,Influenced,Perseverance,DesireToTakeInitiative,Competitiveness,...,ReasonsForLack,Target_ent_competency,NotInterested,ParentalPressure,LackKnowledge,NotAbleFinancialRisk,MentalBlock,RiskAdverse,AcademicPressure,FutureRelocation
0,0,Engineering Sciences,No,19,Male,Yes,No,2,2,3,...,NaN,1,0,0,0,0,0,0,0,0
1,1,Engineering Sciences,Yes,22,Male,No,Yes,3,3,3,...,Just not interested! (Want to work in the corp...,0,1,0,0,0,0,0,0,0
2,2,Engineering Sciences,No,18,Male,Yes,No,3,4,3,...,Not willing to start a venture in India and wa...,0,0,0,0,0,0,0,0,1
3,3,Engineering Sciences,Yes,20,Male,Yes,Yes,3,3,3,...,Not able to take a Financial Risk,0,0,0,0,1,0,0,0,0
4,4,Engineering Sciences,Yes,19,Male,Yes,Yes,2,3,3,...,NaN,1,0,0,0,0,0,0,0,0


In [55]:
# CLEAN!!
# Now we can save the CSV file
df1.to_csv('../data/Entrepreneurial_competency_in_university_students-CLEANED.csv')

In [56]:
# Finally, we can load the data in our MySQL database
# User has to input IP, username, and password

print("LOADING DATASET TO MYSQL DATABASE")
print("---------------------------------")

database_ip       = input("Please input the IP of the database: ")
database_username = input("Please input your username: ")
database_password = getpass.getpass("Please input your password: ")

database_name     = 'entrepreneurship'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password,
                                                      database_ip, database_name), pool_recycle=1, pool_timeout=57600).connect()

df1.to_sql(con=database_connection, name='students', if_exists='append',chunksize=100)

database_connection.close()

LOADING DATASET TO MYSQL DATABASE
---------------------------------
Please input the IP of the database: ironhack.coqickwcxiyv.us-east-1.rds.amazonaws.com
Please input your username: admin
Please input your password: ········
